# A simple Mean Reversion System
Most of the following imports are mainstream, well known Python libraries.

alpha_vantage and fix_yahoo are specialist libraries to download stock data free.  To use alpha_vantage you will need to obtain your own key from the providers https://www.alphavantage.co/

ffn is a specialist library to report trading systems statistics

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
%matplotlib notebook
import pandas as pd
from alpha_vantage.timeseries import TimeSeries
import fix_yahoo_finance as yf
#because the is_list_like is moved to pandas.api.types
pd.core.common.is_list_like = pd.api.types.is_list_like
import ffn
#import pixiedust

ModuleNotFoundError: No module named 'fix_yahoo_finance'

## The following two cells can be used to download stock data and then blanked out again once the data has been saved to csv.



In [ ]:
ts = TimeSeries(key='HK7Q1K6I2EIFKTVL', output_format='pandas')
data, meta_data = ts.get_daily_adjusted(symbol='SPY', outputsize='full')
data = yf.download("SPY", start="2019-01-01", end="2021-03-12")

## The following line is to create split adjusted Open prices since only the adjusted close is provided:

data['Adj_Open']=data.Open*(data.Adj_Close/data.Close)

Useful if you want to test taking signal from previous close and trading at the next open.

In [ ]:
data.rename(columns={'Adj Close': 'Adj_Close'}, inplace=True)
data['Adj_Open']=data.Open*(data.Adj_Close/data.Close)
data.to_csv('spy.csv')
#data.to_csv('../data/Stocks/spy.csv')
#data.head()

Chart the data to check there are no obvious problems with the split adjusted data.

In [ ]:
pricing = pd.read_csv(
    'spy.csv',
    #'../data/Stocks/spy.csv',
    header=0,
    parse_dates=["Date"],
    #index_col=0,
    usecols=['Date','Adj_Open', 'Adj_Close'])
#figure(num=None, figsize=(6, 4), dpi=80, facecolor='w', edgecolor='k')
#plt.plot(pricing.Adj_Open, label='Spy Open')
#plt.plot(pricing.Adj_Close, label='Spy Close')
#plt.xlabel('Date')
#plt.ylabel('Price')
#plt.legend();

## Visual inspection of the DataFrame. 

Adjust to see wider and different selection of rows.  e.g pricing.tail(10) shows 
the final ten rows of data. See 10 Minutes to Pandas https://pandas.pydata.org/pandas-docs/stable/10min.html 
or the Pandas Cheat Sheet https://pandas.pydata.org/Pandas_Cheat_Sheet.pdf

In [ ]:
#pricing.head(),pricing.tail()

## Shift the Adjusted Close down one row 

so that it aligns with the next day's Adjusted Open.  That way we can calculate the signals using the previous day's Close and enter the trade on the next day's Open

In [ ]:
stock=pricing.copy()
stock.Adj_Close=stock.Adj_Close.shift(1)

#stock.tail()

## Create three or more separate blocks of data for testing. 

Reserve one of these for out of sample testing.

In [ ]:
#stock_1=stock.iloc[0:2170].copy()
#stock_2=stock.iloc[2170:4342].copy().reset_index(drop=True)
#stock_3=stock.iloc[4342:6513].copy().reset_index(drop=True) 
#stock_2.head()

## Here is the code for the function which loops through the stock data and executes the trades.  

You can experiment with different parameters such as changing the maximum position size or starting capital.

Experiment with changing references:

1.from both trading and signal generation at the Close; to 

2.signal generation at the Close but trading at the next Open. 

You will not be happy with the result suggesting you may need to automate trading to generate signals and trade intraday so you can execute a trade immediately a signal is given.

In [ ]:
# Trade using a simple mean-reversion strategy
def trade(stock, length):

    temp_dict = {}
    # If window length is 0, algorithm doesn't make sense, so exit
    if length == 0:
        return 0

    # Compute rolling means and rolling standard deviation
    #sma and lma are filters to prevent taking long or short positions against the longer term trend
    rolling_window = stock.Adj_Close.rolling(window=length)
    mu = rolling_window.mean()
    sma = stock.Adj_Close.rolling(window=length*1).mean()
    lma = stock.Adj_Close.rolling(window=length * 10).mean()
    std = rolling_window.std()

    #If you don't use a maximum position size the positions will keep on pyramidding.
    #Set max_position to a high number (1000?) to disable this parameter
    #Need to beware of unintended leverage
    max_position = 1
    percent_per_trade = 1.0

    #Slippage and commission adjustment  - simply reduces equity by a percentage guess
    # a setting of 1 means no slippage, a setting of 0.999 gives 0.1% slippage
    slippage_adj = 1

    # Compute the z-scores for each day using the historical data up to that day
    zscores = (stock.Adj_Close - mu) / std

    # Simulate trading
    # Start with your chosen starting capital and no positions
    money = 10000.00
    position_count = 0

    for i, row in enumerate(stock.itertuples(), 0):

        #set up position size so that each position is a fixed position of your account equity
        equity = money + (stock.Adj_Close[i] * position_count)
        if equity > 0:
            fixed_frac = (equity * percent_per_trade) / stock.Adj_Close[i]
        else:
            fixed_frac = 0
        fixed_frac = int(round(fixed_frac))

        #exit all positions if zscore flips from positive to negative or vice versa without going through
        #the neutral zone
        if i > 0:
            if (zscores[i - 1] > 0.5
                    and zscores[i] < -0.5) or (zscores[i - 1] < -0.5
                                               and zscores[i] > 0.5):

                if position_count > 0:
                    money += position_count * stock.Adj_Close[i] * slippage_adj
                elif position_count < 0:
                    money += position_count * stock.Adj_Close[i] * (
                        1 / slippage_adj)
                position_count = 0

        # Sell short if the z-score is > 1 and if the longer term trend is negative
        if (zscores[i] > 1) & (position_count > max_position * -1) & (sma[i] <
                                                                      lma[i]):

            position_count -= fixed_frac
            money += fixed_frac * stock.Adj_Close[i] * slippage_adj

        # Buy long if the z-score is < 1 and the longer term trend is positive
        elif zscores[i] < -1 and position_count < max_position and sma[i] > lma[i]:

            position_count += fixed_frac
            money -= fixed_frac * stock.Adj_Close[i] * (1 / slippage_adj)

        # Clear positions if the z-score between -.5 and .5
        elif abs(zscores[i]) < 0.5:
            #money += position_count * stock.Adj_Close[i]
            if position_count > 0:
                money += position_count * stock.Adj_Close[i] * slippage_adj
            elif position_count < 0:
                money += position_count * stock.Adj_Close[i] * (
                    1 / slippage_adj)
            position_count = 0

        #fill dictionary with the trading results.
        temp_dict[stock.Date[i]] = [
            stock.Adj_Open[i], stock.Adj_Close[i], mu[i], std[i], zscores[i],
            money, position_count, fixed_frac, sma[i], lma[i]
        ]
    #create a dataframe to return for use in calculating and charting the trading results
    pr = pd.DataFrame(data=temp_dict).T
    pr.index.name = 'Date'
    pr.index = pd.to_datetime(pr.index)
    pr.columns = [
        'Open', 'Close', 'mu', 'std', 'zscores', 'money', 'position_count',
        'fixed_frac', 'sma', 'lma'
    ]
    pr['equity'] = pr.money + (pr.Close * pr.position_count)
    #
    return pr

## The next cell calls the function. 

Experiment with different moving averages by altering the number in brackets.

profit = trade(stock, moving_average) runs the back test with the entire price series
profit = trade(stock_1, moving_average) runs the test using only the first third of the data and so forth through 
stock_2 and stock_3

You will need to amend the function "profit.to_csv" to your own raletive or absolute file address

In [ ]:
moving_average=10
profit = trade(stock, moving_average)
profit.to_csv('mean_reversion_profit.csv')
#profit.to_csv('../data/mean_reversion_profit.csv')

## Inspect the reults data frame

In [ ]:
#profit.head() , profit.tail()

## Create a new dataframe 

which contains trhe equity curve as its only column to feed to FFN the stats library

In [ ]:
series=profit[['equity']].copy()

## Stats and Charts

The following charts and stats are allproduced by the FFN Library

In [ ]:
#stats = series.calc_stats()
#stats.display()
#stats.stats

In [ ]:
#prices =series
#ax = stats.prices.to_drawdown_series().plot(figsize=(15, 8),title='Drawdown')

In [ ]:
#ax = prices.plot(logy=True,figsize=(15, 8),title='Equity Curve')

In [ ]:
#profit['position_count'].diff().plot()
#profit['position_count'].plot()


In [ ]:
#stats.plot()
#stats.plot_correlation()
#st_disp = stats.stats


## Output Telegram Bot files.


In [ ]:
#perf = stats.stats


In [ ]:
import sys, os, inspect
from distutils.util import strtobool
import logbook
import pathlib as pl

from pathlib import Path
import pprint as pp

def get_script_dir(follow_symlinks=True):
    if getattr(sys, 'frozen', False): # py2exe, PyInstaller, cx_Freeze
        path = os.path.abspath(sys.executable)
    else:
        path = inspect.getabsfile(get_script_dir)
    if follow_symlinks:
        path = os.path.realpath(path)
    return os.path.dirname(path)

path = get_script_dir()
#LOCAL_BOT_LIVE_PATH=Path(path).parent / "telegram"
LOCAL_BOT_LIVE_PATH = Path(path) / (Path(path).stem + "_Bot")
sys.path.append(os.path.abspath(LOCAL_BOT_LIVE_PATH))

pp.pprint(f'path={path} bot_path={LOCAL_BOT_LIVE_PATH}')

# Write output exhaust to  Bot dir
profit.to_csv(str(LOCAL_BOT_LIVE_PATH) + '/mr.csv')
# SPY series. May need to be shifted down by one, like profit
data.to_csv(str(LOCAL_BOT_LIVE_PATH) + '/spy.csv')